In [49]:
import nibabel as nb
import numpy as np
import scipy.io
import scipy
from scipy import stats
import math
import pandas as pd
import statsmodels.api as sm

In [2]:
file_path = 'E:/FPP_files_and_codes/FPP_files/relabeled_seconds/corrected_GPT_labeling/final_labels.csv'

RelabelDataframe = pd.read_csv(file_path)

In [3]:
RelabelDictionary = {}
for index, row in RelabelDataframe.iterrows():
    if index != 0:
        RelabelDictionary[row['prv_label']] = row['new_label']
RelabelDictionary[''] = float('nan')

In [4]:
labels_list_1 = []
with open("E:/FPP_files_and_codes/FPP_files/HCP_movie_labels/Movie_labels/Movie1_labels.txt",'r') as file:
    for line in file:
        labels = line.strip().split(',')
        labels_list_1.append(labels)
labels_list_2 = []
with open("E:/FPP_files_and_codes/FPP_files/HCP_movie_labels/Movie_labels/Movie2_labels.txt",'r') as file:
    for line in file:
        labels = line.strip().split(',')
        labels_list_2.append(labels)
labels_list_3 = []
with open("E:/FPP_files_and_codes/FPP_files/HCP_movie_labels/Movie_labels/Movie3_labels.txt",'r') as file:
    for line in file:
        labels = line.strip().split(',')
        labels_list_3.append(labels)
labels_list_4 = []
with open("E:/FPP_files_and_codes/FPP_files/HCP_movie_labels/Movie_labels/Movie4_labels.txt",'r') as file:
    for line in file:
        labels = line.strip().split(',')
        labels_list_4.append(labels)
labels_list = labels_list_1 + labels_list_2 + labels_list_3  + labels_list_4
print(len(labels_list))

3655


In [5]:
RelabeldFrames = [set() for i in range(3655)]
for i in range(3655):
    for j in labels_list[i]:
        RelabeldFrames[i].add(str(RelabelDictionary[j]))

In [6]:
RelabeldFrames

[{'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'nan'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Body', 'Object', 'Place', 'Unclear'},
 {'Body', 'Object', 'Place', 'Unclear'},
 {'Body', 'Object', 'Place', 'Unclear'},
 {'Body', 'Object', 'Place', 'Unclear'},
 {'Object', 'Place'},
 {'Object', 'Place'},
 {'Body', 'Object', 'Place', 'Unclear'},
 {'Body', 'Object', 'Place', 'Unclear'},
 {'B

In [10]:
regressor_animacy = np.zeros(3655)
for i,labels in enumerate(RelabeldFrames):
    if ('Animal' in labels) or ('Face' in labels) or ('Body' in labels):
        regressor_animacy[i] = 1
    else:
        regressor_animacy[i] = -1

In [11]:
regressor_animacy

array([-1., -1., -1., ..., -1., -1., -1.])

In [12]:
regressor_mean_magnitude_motion_vectors = np.load('D:/place_ROIs_project/Phase5_AnalysisBasedOnPickActivityEachPatch/MotionBrainMaps/MotionMeanMap.npy')

In [13]:
shift = 5
regressor_animacy_shifted = np.roll(regressor_animacy, shift)
regressor_mean_magnitude_motion_vectors_shifted = np.roll(regressor_mean_magnitude_motion_vectors, shift)

In [14]:
regressor_animate_motion = regressor_animacy_shifted * regressor_mean_magnitude_motion_vectors_shifted

In [20]:
regressor_animate_motion[regressor_animate_motion == -0.0] = 0.0

In [21]:
regressor_animate_motion

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:

def compute_correlation_and_save_maps_right(data, temp_r_path, output_corr_path, output_signif_path):
    # Load data
    right_data = scipy.io.loadmat('E:/FPP_files_and_codes/FPP_files/Movie_watching_avg_data/notmean_right_Meanfile.mat')['Right_data']
    
    # Initialize arrays for correlation and p-values
    corr_right = np.zeros(32492)
    pvalue_right = np.zeros(32492)
    
    # Compute correlation and p-values
    for i in range(32492):
        corr, p_value = stats.pearsonr(data, right_data[i])
        corr_right[i] = corr
        pvalue_right[i] = p_value
    
    # Handle NaNs and convert to float32
    corr_right = np.nan_to_num(corr_right)
    pvalue_right = np.nan_to_num(pvalue_right)
    corr_right_32 = corr_right.astype(np.float32)
    pvalue_right_32 = pvalue_right.astype(np.float32)
    
    # Load template, set correlation data, and save
    temp_r = nb.load(temp_r_path)
    temp_r.darrays[0].data = corr_right_32
    temp_r.to_filename(output_corr_path)
    
    # Compute significance map
    significance_right = -1 * np.log(pvalue_right) * np.sign(corr_right)
    significance_right = np.nan_to_num(significance_right, nan=0.0, posinf=0.0, neginf=0.0)
    significance_right_32 = significance_right.astype(np.float32)
    
    # Load template, set significance data, and save
    temp_r = nb.load(temp_r_path)
    temp_r.darrays[0].data = significance_right_32
    temp_r.to_filename(output_signif_path)

In [22]:
left_data = scipy.io.loadmat('E:/FPP_files_and_codes/FPP_files/Movie_watching_avg_data/notmean_left_Meanfile.mat')['Left_data']
right_data = scipy.io.loadmat('E:/FPP_files_and_codes/FPP_files/Movie_watching_avg_data/notmean_right_Meanfile.mat')['Right_data']

In [33]:
corr_right = np.zeros(32492)
pvalue_right = np.zeros(32492)
for i in range (32492):
    corr,p_value = stats.pearsonr(regressor_animate_motion, right_data[i])
    corr_right[i] = corr
    pvalue_right[i] = p_value

C:\Users\roza\AppData\Local\Temp\ipykernel_107924\2910972520.py:4: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr,p_value = stats.pearsonr(regressor_animate_motion, right_data[i])


In [34]:
corr_right = np.nan_to_num(corr_right)
pvalue_right = np.nan_to_num(pvalue_right)
corr_right_32 = corr_right.astype(np.float32)
pvalue_right_32 = pvalue_right.astype(np.float32)

In [35]:
temp_r_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.R.func.gii'
temp_r = nb.load(temp_r_path)
temp_r.darrays[0].data = corr_right_32
temp_r.to_filename('E:/FPP_files_and_codes/FPP_files/motion_maps/animate_motion/correlation_regressor_animate_motion.R.func.gii')

In [36]:
significance_right = -1 * np.log(pvalue_right) * np.sign(corr_right)

C:\Users\roza\AppData\Local\Temp\ipykernel_107924\2960793998.py:1: RuntimeWarning: divide by zero encountered in log
  significance_right = -1 * np.log(pvalue_right) * np.sign(corr_right)
C:\Users\roza\AppData\Local\Temp\ipykernel_107924\2960793998.py:1: RuntimeWarning: invalid value encountered in multiply
  significance_right = -1 * np.log(pvalue_right) * np.sign(corr_right)


In [37]:
significance_right = np.nan_to_num(significance_right, nan=0.0, posinf=0.0, neginf=0.0)
significance_right_32 = significance_right.astype(np.float32)

In [38]:
temp_r_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.R.func.gii'
temp_r = nb.load(temp_r_path)
temp_r.darrays[0].data = significance_right_32
temp_r.to_filename('E:/FPP_files_and_codes/FPP_files/motion_maps/animate_motion/significance_correlation_regressor_animate_motion.R.func.gii')

In [39]:
corr_left = np.zeros(32492)
pvalue_left = np.zeros(32492)
for i in range (32492):
    corr,p_value = stats.pearsonr(regressor_animate_motion, left_data[i])
    corr_left[i] = corr
    pvalue_left[i] = p_value

C:\Users\roza\AppData\Local\Temp\ipykernel_107924\2159100098.py:4: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr,p_value = stats.pearsonr(regressor_animate_motion, left_data[i])


In [40]:
corr_left = np.nan_to_num(corr_left)
pvalue_left = np.nan_to_num(pvalue_left)
corr_left_32 = corr_left.astype(np.float32)
pvalue_left_32 = pvalue_left.astype(np.float32)

In [41]:
temp_l_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.L.func.gii'
temp_l = nb.load(temp_l_path)
temp_l.darrays[0].data = corr_left_32
temp_l.to_filename('E:/FPP_files_and_codes/FPP_files/motion_maps/animate_motion/correlation_regressor_animate_motion.L.func.gii')

In [42]:
significance_left = -1 * np.log(pvalue_left) * np.sign(corr_left)

C:\Users\roza\AppData\Local\Temp\ipykernel_107924\2113548353.py:1: RuntimeWarning: divide by zero encountered in log
  significance_left = -1 * np.log(pvalue_left) * np.sign(corr_left)
C:\Users\roza\AppData\Local\Temp\ipykernel_107924\2113548353.py:1: RuntimeWarning: invalid value encountered in multiply
  significance_left = -1 * np.log(pvalue_left) * np.sign(corr_left)


In [43]:
significance_left = np.nan_to_num(significance_left, nan=0.0, posinf=0.0, neginf=0.0)
significance_left_32 = significance_left.astype(np.float32)

In [44]:
temp_l_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.L.func.gii'
temp_l = nb.load(temp_l_path)
temp_l.darrays[0].data = significance_left_32
temp_l.to_filename('E:/FPP_files_and_codes/FPP_files/motion_maps/animate_motion/significance_correlation_regressor_animate_motion.L.func.gii')

# regress out the effect of motion and animacy

In [51]:
regressor_mean_magnitude_motion_vectors = np.load('D:/place_ROIs_project/Phase5_AnalysisBasedOnPickActivityEachPatch/MotionBrainMaps/MotionMeanMap.npy')
shifted_regressor_mean_magnitude_motion_vectors = np.roll(regressor_mean_magnitude_motion_vectors, delay)

In [52]:
shifted_regressor_mean_magnitude_motion_vectors_2d = shifted_regressor_mean_magnitude_motion_vectors.reshape(1, -1)
shifted_regressor_animacy_2d = regressor_animacy_shifted.reshape(1, -1)
shifted_regressor_animate_motion_2d = regressor_animate_motion.reshape(1,-1)
X = np.concatenate((shifted_regressor_animate_motion_2d, shifted_regressor_animacy_2d, shifted_regressor_mean_magnitude_motion_vectors_2d), axis=0).T

coefficients_regressor_motion_right = np.zeros(32492)
coefficients_regrssor_animacy_right = np.zeros(32492)
coefficients_regrssor_animate_motion_right = np.zeros(32492)

pvalue_coefficients_regressor_motion_right = np.zeros(32492)
pvalue_coefficients_regrssor_animacy_right = np.zeros(32492)
pvalue_coefficients_regrssor_animate_motion_right = np.zeros(32492)

zero = np.zeros(3655)

for (i,vertex_data) in enumerate(right_data):
    y = vertex_data
    df = pd.DataFrame(X, columns=['X1', 'X2', 'X3'])
    df['y'] = y

    # Fit GLM model
    X = sm.add_constant(df[['X1', 'X2', 'X3']]) 
    model = sm.GLM(df['y'], X, family=sm.families.Gaussian())
    if not np.array_equal(vertex_data, zero):
        result = model.fit()
        coefficients = result.params

        coefficients_regrssor_animate_motion_right[i] = coefficients['X1']
        coefficients_regrssor_animacy_right[i] = coefficients['X2']
        coefficients_regressor_motion_right[i] = coefficients['X3']

        p_values = result.pvalues
        pvalue_coefficients_regrssor_animate_motion_right[i] = p_values['X1']
        pvalue_coefficients_regrssor_animacy_right[i] = p_values['X2']
        pvalue_coefficients_regressor_motion_right[i] = p_values['X3']

In [53]:
def save_beta_and_significance_map_right(coefficients, pvalues, path_beta_map, path_significance_map):
    coefficients = np.nan_to_num(coefficients)
    pvalues = np.nan_to_num(pvalues)
    coefficients_32 = coefficients.astype(np.float32)
    pvalue_32 = pvalues.astype(np.float32)
    temp_r_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.R.func.gii'
    temp_r = nb.load(temp_r_path)
    temp_r.darrays[0].data = coefficients_32
    temp_r.to_filename(path_beta_map)

    significance_beta = -1* np.log(pvalues) * np.sign(coefficients)
    significance_beta = np.nan_to_num(significance_beta, nan=0.0, posinf=0.0, neginf=0.0)
    float32_significance_beta = significance_beta.astype(np.float32)
    temp_r_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.R.func.gii'
    temp_r = nb.load(temp_r_path)
    temp_r.darrays[0].data = float32_significance_beta
    temp_r.to_filename(path_significance_map)

In [54]:
save_beta_and_significance_map_right(coefficients_regrssor_animate_motion_right, pvalue_coefficients_regrssor_animate_motion_right, \
                                    'E:/FPP_files_and_codes/FPP_files/motion_maps/animate_motion_beta_regressout_motion_and_animacy/beta_regressor_animate_motion.R.func.gii',\
                                    'E:/FPP_files_and_codes/FPP_files/motion_maps/animate_motion_beta_regressout_motion_and_animacy/significance_beta_regressor_animate_motion.R.func.gii')

save_beta_and_significance_map_right(coefficients_regrssor_animacy_right, pvalue_coefficients_regrssor_animacy_right, \
                                    'E:/FPP_files_and_codes/FPP_files/motion_maps/animate_motion_beta_regressout_motion_and_animacy/beta_regrssor_animacy.R.func.gii',\
                                    'E:/FPP_files_and_codes/FPP_files/motion_maps/animate_motion_beta_regressout_motion_and_animacy/significance_beta_regrssor_animacy.R.func.gii')

save_beta_and_significance_map_right(coefficients_regressor_motion_right, pvalue_coefficients_regressor_motion_right, \
                                    'E:/FPP_files_and_codes/FPP_files/motion_maps/animate_motion_beta_regressout_motion_and_animacy/beta_regressor_motion.R.func.gii',\
                                    'E:/FPP_files_and_codes/FPP_files/motion_maps/animate_motion_beta_regressout_motion_and_animacy/significance_beta_regressor_motion.R.func.gii')


C:\Users\roza\AppData\Local\Temp\ipykernel_107924\2976885400.py:11: RuntimeWarning: divide by zero encountered in log
  significance_beta = -1* np.log(pvalues) * np.sign(coefficients)
C:\Users\roza\AppData\Local\Temp\ipykernel_107924\2976885400.py:11: RuntimeWarning: invalid value encountered in multiply
  significance_beta = -1* np.log(pvalues) * np.sign(coefficients)


In [55]:
shifted_regressor_mean_magnitude_motion_vectors_2d = shifted_regressor_mean_magnitude_motion_vectors.reshape(1, -1)
shifted_regressor_animacy_2d = regressor_animacy_shifted.reshape(1, -1)
shifted_regressor_animate_motion_2d = regressor_animate_motion.reshape(1,-1)
X = np.concatenate((shifted_regressor_animate_motion_2d, shifted_regressor_animacy_2d, shifted_regressor_mean_magnitude_motion_vectors_2d), axis=0).T

coefficients_regressor_motion_left = np.zeros(32492)
coefficients_regrssor_animacy_left = np.zeros(32492)
coefficients_regrssor_animate_motion_left = np.zeros(32492)

pvalue_coefficients_regressor_motion_left = np.zeros(32492)
pvalue_coefficients_regrssor_animacy_left = np.zeros(32492)
pvalue_coefficients_regrssor_animate_motion_left = np.zeros(32492)

zero = np.zeros(3655)

for (i,vertex_data) in enumerate(left_data):
    y = vertex_data
    df = pd.DataFrame(X, columns=['X1', 'X2', 'X3'])
    df['y'] = y

    # Fit GLM model
    X = sm.add_constant(df[['X1', 'X2', 'X3']]) 
    model = sm.GLM(df['y'], X, family=sm.families.Gaussian())
    if not np.array_equal(vertex_data, zero):
        result = model.fit()
        coefficients = result.params

        coefficients_regrssor_animate_motion_left[i] = coefficients['X1']
        coefficients_regrssor_animacy_left[i] = coefficients['X2']
        coefficients_regressor_motion_left[i] = coefficients['X3']

        p_values = result.pvalues
        pvalue_coefficients_regrssor_animate_motion_left[i] = p_values['X1']
        pvalue_coefficients_regrssor_animacy_left[i] = p_values['X2']
        pvalue_coefficients_regressor_motion_left[i] = p_values['X3']

In [56]:
def save_beta_and_significance_map_left(coefficients, pvalues, path_beta_map, path_significance_map):
    coefficients = np.nan_to_num(coefficients)
    pvalues = np.nan_to_num(pvalues)
    coefficients_32 = coefficients.astype(np.float32)
    pvalue_32 = pvalues.astype(np.float32)
    temp_l_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.L.func.gii'
    temp_l = nb.load(temp_l_path)
    temp_l.darrays[0].data = coefficients_32
    temp_l.to_filename(path_beta_map)

    significance_beta = -1* np.log(pvalues) * np.sign(coefficients)
    significance_beta = np.nan_to_num(significance_beta, nan=0.0, posinf=0.0, neginf=0.0)
    float32_significance_beta = significance_beta.astype(np.float32)
    temp_l_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.L.func.gii'
    temp_l = nb.load(temp_l_path)
    temp_l.darrays[0].data = float32_significance_beta
    temp_l.to_filename(path_significance_map)

In [57]:
save_beta_and_significance_map_left(coefficients_regrssor_animate_motion_left, pvalue_coefficients_regrssor_animate_motion_left, \
                                    'E:/FPP_files_and_codes/FPP_files/motion_maps/animate_motion_beta_regressout_motion_and_animacy/beta_regressor_animate_motion.L.func.gii',\
                                    'E:/FPP_files_and_codes/FPP_files/motion_maps/animate_motion_beta_regressout_motion_and_animacy/significance_beta_regressor_animate_motion.L.func.gii')

save_beta_and_significance_map_left(coefficients_regrssor_animacy_left, pvalue_coefficients_regrssor_animacy_left, \
                                    'E:/FPP_files_and_codes/FPP_files/motion_maps/animate_motion_beta_regressout_motion_and_animacy/beta_regrssor_animacy.L.func.gii',\
                                    'E:/FPP_files_and_codes/FPP_files/motion_maps/animate_motion_beta_regressout_motion_and_animacy/significance_beta_regrssor_animacy.L.func.gii')

save_beta_and_significance_map_left(coefficients_regressor_motion_left, pvalue_coefficients_regressor_motion_left, \
                                    'E:/FPP_files_and_codes/FPP_files/motion_maps/animate_motion_beta_regressout_motion_and_animacy/beta_regressor_motion.L.func.gii',\
                                    'E:/FPP_files_and_codes/FPP_files/motion_maps/animate_motion_beta_regressout_motion_and_animacy/significance_beta_regressor_motion.L.func.gii')


C:\Users\roza\AppData\Local\Temp\ipykernel_107924\2622879532.py:11: RuntimeWarning: divide by zero encountered in log
  significance_beta = -1* np.log(pvalues) * np.sign(coefficients)
C:\Users\roza\AppData\Local\Temp\ipykernel_107924\2622879532.py:11: RuntimeWarning: invalid value encountered in multiply
  significance_beta = -1* np.log(pvalues) * np.sign(coefficients)
